In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('../..')

In [3]:
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

In [4]:
from stack_segmentation.stack import Stack

In [5]:
from stack_segmentation.aug_pipelines import medium_aug
from stack_segmentation.io import make_dataloader, collate_fn_basic
from stack_segmentation.training import handle_stacks_data, make_model, train_loop
from stack_segmentation.unet import UNet
from stack_segmentation.pipeline_config import dataloaders_conf, model_conf, train_conf

In [6]:
from exp_config import data_conf

## Parameters to tune

In [7]:
# train_conf['device'] = 'cpu'
# model_conf['device'] = 'cpu'

In [8]:
data_conf['conf_name'] = 'basic'
data_conf

{'conf_name': 'basic',
 'stacks': [{'path': '../../data/carb96558',
   'slice_train': (slice(None, None, None),
    slice(None, None, None),
    slice(None, 230, None)),
   'slice_val': (slice(None, None, None),
    slice(None, None, None),
    slice(250, 470, None))},
  {'path': '../../data/SoilB-2',
   'slice_train': (slice(None, None, None),
    slice(None, None, None),
    slice(None, 230, None)),
   'slice_val': (slice(None, None, None),
    slice(None, None, None),
    slice(240, 460, None))},
  {'path': '../../data/Urna_22',
   'slice_train': (slice(None, None, None),
    slice(None, None, None),
    slice(None, 220, None)),
   'slice_val': (slice(None, None, None),
    slice(None, None, None),
    slice(245, 455, None))},
  {'path': '../../data/carb96558',
   'slice_test': (slice(None, None, None),
    slice(None, None, None),
    slice(490, None, None))},
  {'path': '../../data/carb71',
   'slice_test': (slice(None, None, None),
    slice(None, None, None),
    slice(None, Non

In [9]:
dataloaders_conf['train']['augmentation_pipeline'] = None
dataloaders_conf

{'train': {'batch_size': 32,
  'num_workers': 8,
  'shuffle': True,
  'augmentation_pipeline': None},
 'val': {'batch_size': 32,
  'num_workers': 8,
  'shuffle': False,
  'augmentation_pipeline': None},
 'test': {'batch_size': 32,
  'num_workers': 8,
  'shuffle': True,
  'augmentation_pipeline': None}}

In [10]:
train_conf

{'num_epochs': 200, 'device': 'cuda:0'}

In [11]:
model_conf['opt_type'] = 'SGD'
model_conf['loss'] = [('BCE', 1)]
model_conf['weight'] = None
model_conf['nesterov'] = True
model_conf

{'device': 'cuda:0',
 'weight': None,
 'loss': [('BCE', 1)],
 'opt_type': 'SGD',
 'lr': 0.0001,
 'weight_decay': 0.0001,
 'amsgrad': False,
 'nesterov': True,
 'momentum': 0.9,
 'centered': False,
 'min_lr': 1e-06,
 'factor': 0.5,
 'patience': 5}

In [12]:
data_conf['patches']

{'train': (128, 128, 1), 'val': (128, 128, 1), 'test': (128, 128, 1)}

## Prepare train, validation and test data

In [13]:
data_train, data_val, data_test = handle_stacks_data(**data_conf)

720it [00:01, 702.53it/s]
100%|██████████| 720/720 [00:07<00:00, 93.06it/s] 
8280it [00:00, 233936.69it/s]
7920it [00:00, 228778.64it/s]
700it [00:00, 889.57it/s]
100%|██████████| 700/700 [00:06<00:00, 100.06it/s]
8280it [00:00, 231944.63it/s]
7920it [00:00, 81301.67it/s]
710it [00:00, 864.26it/s]
100%|██████████| 710/710 [00:07<00:00, 96.99it/s] 
7920it [00:00, 250935.85it/s]
7560it [00:00, 233328.95it/s]
720it [00:00, 808.64it/s]
100%|██████████| 720/720 [00:07<00:00, 94.47it/s] 
8280it [00:00, 243576.10it/s]
720it [00:00, 824.63it/s]
100%|██████████| 720/720 [00:07<00:00, 94.51it/s] 
25920it [00:00, 118795.44it/s]
700it [00:00, 853.71it/s]
100%|██████████| 700/700 [00:07<00:00, 99.16it/s] 
25200it [00:00, 128942.15it/s]
509it [00:00, 1204.23it/s]
100%|██████████| 509/509 [00:02<00:00, 198.35it/s]
8144it [00:00, 228473.66it/s]
700it [00:00, 830.61it/s]
100%|██████████| 700/700 [00:07<00:00, 98.17it/s] 
25200it [00:00, 230162.69it/s]
700it [00:00, 836.82it/s]
100%|██████████| 700/700 

In [14]:
len(data_train), len(data_val), len(data_test)

(24480, 23400, 11)

In [15]:
dataloader_train = make_dataloader(
    samples=data_train, 
    collate_fn=collate_fn_basic,
    **dataloaders_conf['train']
)

dataloader_val = make_dataloader(
    samples=data_val, 
    collate_fn=collate_fn_basic,
    **dataloaders_conf['val']
)

dataloaders_test = {
    name: make_dataloader(
        samples=data, 
        collate_fn=collate_fn_basic,
        **dataloaders_conf['test']
    ) for name, data in data_test.items()}

## Create model and metrics

In [ ]:
from stack_segmentation.metrics import accuracy, precision, recall, f1, pr_auc, iou

In [ ]:
metrics = {
    'accuracy': accuracy, 
    'precision': precision, 
    'recall': recall, 
    'f1': f1,
    'pr_auc': pr_auc, 
    'iou': iou,
}

In [ ]:
model, criterion, optimizer, scheduler = make_model(**model_conf)

## Run experiment

In [ ]:
results = train_loop(
    model=model,
    dataloader_train=dataloader_train, 
    dataloader_val=dataloader_val,
    dataloaders_test=dataloaders_test,
    criterion=criterion, 
    optimizer=optimizer, 
    scheduler=scheduler,
    metrics=metrics,
    exp_name=data_conf['conf_name'],
    **train_conf)

  0%|          | 0/765 [00:00<?, ?it/s]

Epoch 0...


  0%|          | 0/732 [00:00<?, ?it/s]

Mean train loss: 0.56711


  0%|          | 0/765 [00:00<?, ?it/s]

...

100%|██████████| 732/732 [00:57<00:00, 12.82it/s]


Mean val loss: 0.018292


  0%|          | 0/765 [00:00<?, ?it/s]

Epoch 84...


  5%|▌         | 39/765 [00:11<02:28,  4.90it/s]

## Dump experiment results

In [ ]:
import pickle
import json

In [ ]:
p = './{}_exp_results.pkl'.format(data_conf['conf_name'])
with open(p, 'wb') as f:
    pickle.dump(results, f)

In [17]:
# p = './{}_exp_results.pkl'.format(data_conf['conf_name'])
# with open(p, 'rb') as f:
#     results = pickle.load(f)

In [11]:
import torch
model.load_state_dict(torch.load('./{}.pt'.format(data_conf['conf_name'])))

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

## Train and validation losses

In [ ]:
from itertools import chain

In [ ]:
train_losses = list(chain(*[item for item in results['train_losses']]))
val_losses = list(chain(*[item for item in results['val_losses']]))

In [ ]:
def moving_average(a, n=5) :
    ret = np.cumsum([a[0]] * (n - 1) + a, dtype=float)
    ret[n:] = ret[n:] - ret[:-n]
    return ret[n - 1:] / n

In [ ]:
plt.figure(figsize=(10, 10))
plt.title('Moving-averaged batch losses')
plt.plot(np.arange(len(train_losses)), moving_average(train_losses), label='train')
plt.plot(np.arange(len(val_losses)), moving_average(val_losses), label='validation')

plt.legend(loc='best')
plt.yscale('log')

# plt.ylim([1e-2, 1])
plt.show()

In [ ]:
mean_train_loss = [np.mean(item) for item in results['train_losses']]
mean_val_loss = [np.mean(item) for item in results['val_losses']]

In [ ]:
plt.figure(figsize=(10, 10))
plt.title('Epoch losses')
plt.plot(np.arange(len(mean_train_loss)) + 1, mean_train_loss, label='train')
plt.plot(np.arange(len(mean_val_loss)) + 1, mean_val_loss, label='val')

plt.yscale('log')
plt.legend(loc='best')

plt.xlim([1, len(mean_train_loss) + 1])
plt.show()

## Results